In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import torch

from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task
import lightautoml


In [ ]:
train = pd.read_csv('../input/playground-series-s4e7/train.csv')
test = pd.read_csv('../input/playground-series-s4e7/test.csv')
ss = pd.read_csv('../input/playground-series-s4e7/sample_submission.csv')

In [ ]:
HOLDOUT = False

In [ ]:
if HOLDOUT:
    X_train, X_val = train_test_split(train, test_size=0.2, random_state=42, shuffle=True, stratify=train.Response)
else:
    X_train, X_val = train, test

In [ ]:

task = Task('binary') #‘binary’ 
automl = TabularAutoML(
    task = task,
    timeout = 600 * 3600,
    cpu_limit = 12,
    general_params = {'use_algos':[['lgb']]},
    selection_params ={'mode' : 0},
    tuning_params = {'max_tuning_time': 0},
    reader_params = {'n_jobs': 12, 'cv': 5, 'random_state': 42}
)

out_of_fold_predictions1 = automl.fit_predict(
    X_train,
    roles = {
        'target': 'Response',
        'drop': ['id']
    }, 
    verbose = 4
)

val_predict = automl.predict(X_val)
if HOLDOUT:
    roc_auc_score(X_val.Response, val_predict.data)

In [ ]:
roc_auc_score(X_train.Response, out_of_fold_predictions1.data[:, 0])

## 2 binary model

In [ ]:

task = Task('binary') #‘binary’ 
automl1 = TabularAutoML(
    task = task,
    timeout = 600 * 3600,
    cpu_limit = 12,
    general_params = {'use_algos':[['lgb']]},
    selection_params ={'mode' : 0},
    tuning_params = {'max_tuning_time': 0},
    reader_params = {'n_jobs': 12, 'cv': 5, 'random_state': 42}
)

out_of_fold_predictions1_1 = automl1.fit_predict(
    X_train[X_train.Previously_Insured==0], 
    roles = {
        'target': 'Response',
        'drop': ['id']
    }, 
    verbose = 4
)


automl2 = TabularAutoML(
    task = task,
    timeout = 600 * 3600,
    cpu_limit = 12,
    general_params = {'use_algos':[['lgb']]},
    selection_params ={'mode' : 0},
    tuning_params = {'max_tuning_time': 0},
    reader_params = {'n_jobs': 12, 'cv': 5, 'random_state': 42}
)
out_of_fold_predictions2_1 = automl2.fit_predict(
    X_train[X_train.Previously_Insured==1],
    roles = {
        'target': 'Response',
        'drop': ['id']
    }, 
    verbose = 4
)

val_predict1 = automl1.predict(X_val[X_val.Previously_Insured==0])
val_predict2 = automl2.predict(X_val[X_val.Previously_Insured==1])


y_pred = pd.Series(index=X_val.index)
y_pred.iloc[X_val.Previously_Insured==0] = val_predict1.data[:, 0]
y_pred.iloc[X_val.Previously_Insured==1] = val_predict2.data[:, 0]

oof1 = pd.Series(index=X_train.index)
oof1.iloc[X_train.Previously_Insured==0] = out_of_fold_predictions1_1.data[:, 0]
oof1.iloc[X_train.Previously_Insured==1] = out_of_fold_predictions2_1.data[:, 0]

if HOLDOUT:
    roc_auc_score(X_val.Response, y_pred)

In [ ]:
roc_auc_score(X_train.Response, oof1)

In [ ]:

task = Task('binary') #‘binary’ 
automl1_1 = TabularAutoML(
    task = task,
    timeout = 600 * 3600,
    cpu_limit = 12,
    general_params = {'use_algos':[['lgb']]},
    selection_params ={'mode' : 0},
    tuning_params = {'max_tuning_time': 0},
    reader_params = {'n_jobs': 12, 'cv': 5, 'random_state': 42}
)

out_of_fold_predictions1_2 = automl1_1.fit_predict(
    X_train[X_train.Vehicle_Damage=='Yes'], 
    roles = {
        'target': 'Response',
        'drop': ['id']
    }, 
    verbose = 4
)


automl2_1 = TabularAutoML(
    task = task,
    timeout = 600 * 3600,
    cpu_limit = 12,
    general_params = {'use_algos':[['lgb']]},
    selection_params ={'mode' : 0},
    tuning_params = {'max_tuning_time': 0},
    reader_params = {'n_jobs': 12, 'cv': 5, 'random_state': 42}
)
out_of_fold_predictions2_2 = automl2_1.fit_predict(
    X_train[X_train.Vehicle_Damage=='No'],
    roles = {
        'target': 'Response',
        'drop': ['id']
    }, 
    verbose = 4
)

val_predict1_2 = automl1_1.predict(X_val[X_val.Vehicle_Damage=='Yes'])
val_predict2_2 = automl2_1.predict(X_val[X_val.Vehicle_Damage=='No'])


y_pred2 = pd.Series(index=X_val.index)
y_pred2.iloc[X_val.Vehicle_Damage=='Yes'] = val_predict1_2.data[:, 0]
y_pred2.iloc[X_val.Vehicle_Damage=='No'] = val_predict2_2.data[:, 0]

oof2 = pd.Series(index=X_train.index)
oof2.iloc[X_train.Vehicle_Damage=='Yes'] = out_of_fold_predictions1_2.data[:, 0]
oof2.iloc[X_train.Vehicle_Damage=='No'] = out_of_fold_predictions2_2.data[:, 0]

if HOLDOUT:
    roc_auc_score(X_val.Response, y_pred2)

In [ ]:
roc_auc_score(X_train.Response, oof2)

In [ ]:
if HOLDOUT:
    roc_auc_score(X_val.Response, y_pred+y_pred2)

In [ ]:
roc_auc_score(X_train.Response, oof1+oof2)

# 4 binary models 

In [ ]:
Xt1 = X_train[X_train.Previously_Insured==0]
Xt1 = Xt1[Xt1.Vehicle_Damage=='Yes']

Xt2 = X_train[X_train.Previously_Insured==0]
Xt2 = Xt2[Xt2.Vehicle_Damage=='No']

Xt3 = X_train[X_train.Previously_Insured==1]
Xt3 = Xt3[Xt3.Vehicle_Damage=='Yes']

Xt4 = X_train[X_train.Previously_Insured==1]
Xt4 = Xt4[Xt4.Vehicle_Damage=='No']

Xv1 = X_val[X_val.Previously_Insured==0]
Xv1 = Xv1[Xv1.Vehicle_Damage=='Yes']

Xv2 = X_val[X_val.Previously_Insured==0]
Xv2 = Xv2[Xv2.Vehicle_Damage=='No']

Xv3 = X_val[X_val.Previously_Insured==1]
Xv3 = Xv3[Xv3.Vehicle_Damage=='Yes']

Xv4 = X_val[X_val.Previously_Insured==1]
Xv4 = Xv4[Xv4.Vehicle_Damage=='No']

task = Task('binary') #‘binary’ 

automl1_3 = TabularAutoML(
    task = task,
    timeout = 600 * 3600,
    cpu_limit = 12,
    general_params = {'use_algos':[['lgb']]},
    selection_params ={'mode' : 0},
    tuning_params = {'max_tuning_time': 0},
    reader_params = {'n_jobs': 12, 'cv': 5, 'random_state': 42}
)

out_of_fold_predictions1_3 = automl1_3.fit_predict(
    Xt1, 
    roles = {
        'target': 'Response',
        'drop': ['id']
    }, 
    verbose = 4
)

automl2_3 = TabularAutoML(
    task = task,
    timeout = 600 * 3600,
    cpu_limit = 12,
    general_params = {'use_algos':[['lgb']]},
    selection_params ={'mode' : 0},
    tuning_params = {'max_tuning_time': 0},
    reader_params = {'n_jobs': 12, 'cv': 5, 'random_state': 42}
)

out_of_fold_predictions2_3 = automl2_3.fit_predict(
    Xt2, 
    roles = {
        'target': 'Response',
        'drop': ['id']
    }, 
    verbose = 4
)

automl3_3 = TabularAutoML(
    task = task,
    timeout = 600 * 3600,
    cpu_limit = 12,
    general_params = {'use_algos':[['lgb']]},
    selection_params ={'mode' : 0},
    tuning_params = {'max_tuning_time': 0},
    reader_params = {'n_jobs': 12, 'cv': 5, 'random_state': 42}
)

out_of_fold_predictions3_3 = automl3_3.fit_predict(
    Xt3,
    roles = {
        'target': 'Response',
        'drop': ['id']
    }, 
    verbose = 4
)

automl4_3 = TabularAutoML(
    task = task,
    timeout = 600 * 3600,
    cpu_limit = 12,
    general_params = {'use_algos':[['lgb']]},
    selection_params ={'mode' : 0},
    tuning_params = {'max_tuning_time': 0},
    reader_params = {'n_jobs': 12, 'cv': 5, 'random_state': 42}
)

out_of_fold_predictions4_3 = automl4_3.fit_predict(
    Xt4,
    roles = {
        'target': 'Response',
        'drop': ['id']
    }, 
    verbose = 4
)

val_predict1_3 = automl1_3.predict(Xv1)
val_predict2_3 = automl2_3.predict(Xv2)
val_predict3_3 = automl3_3.predict(Xv3)
val_predict4_3 = automl4_3.predict(Xv4)

y_pred3 = pd.Series(index=X_val.index)
y_pred3.loc[Xv1.index] = val_predict1_3.data[:, 0]
y_pred3.loc[Xv2.index] = val_predict2_3.data[:, 0]
y_pred3.loc[Xv3.index] = val_predict3_3.data[:, 0]
y_pred3.loc[Xv4.index] = val_predict4_3.data[:, 0]

oof3 = pd.Series(index=X_train.index)
oof3.loc[Xt1.index] = out_of_fold_predictions1_3.data[:, 0]
oof3.loc[Xt2.index] = out_of_fold_predictions2_3.data[:, 0]
oof3.loc[Xt3.index] = out_of_fold_predictions3_3.data[:, 0]
oof3.loc[Xt4.index] = out_of_fold_predictions4_3.data[:, 0]


if HOLDOUT:
    roc_auc_score(X_val.Response, y_pred3.values)

In [ ]:
roc_auc_score(X_train.Response, oof3)

In [ ]:
import joblib

In [ ]:
joblib.dump(
    {'m1_full_train': automl,
    'm2': {'Previously_Insured0': automl1, 'Previously_Insured1': automl2},
    'm3': {'Vehicle_DamageY': automl1_1, 'Vehicle_DamageN':automl2_1},
    'm4': {'PI0VDY': automl1_3, 'PI0VDN':automl2_3, 'PI1VDY':automl3_3, 'PI1VDN':automl4_3}},
     'gbdts_blend.jbl')

In [ ]:
# oofs_all = {'m1': out_of_fold_predictions1.data[:, 0], 'm2': oof1, 'm3': oof2, 'm4': oof3}
# preds_all = {'m1': val_predict.data[:, 0], 'm2': y_pred.values, 'm3': y_pred2.values, 'm4': y_pred3.values}

In [ ]:
# joblib.dump(oofs_all, 'oof.jbl')

In [ ]:
# joblib.dump(preds_all, 'pred.jbl')